In [5]:
!pip install openai langchain pandas
!pip install langchain-community langchain-core
!pip install --upgrade langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [6]:
import os
import json
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [21]:
# Set your OpenAI key
os.environ["OPENAI_API_KEY"] = ""


In [22]:
# Define Product Filter Function (Tool)
import json

def load_catalog():
    with open("/content/catalog.json", "r") as f:
        return json.load(f)


# Tool 1: Filter products
def filter_products_tool(user_input: str):
    catalog = load_catalog()
    # Hardcoded demo filter based on test prompt
    max_price = 5000
    size = 9
    brands = ["Nike", "Adidas"]

    filtered = []
    for item in catalog:
        if (
            item["price"] <= max_price and
            size in item["size_available"] and
            item["brand"].lower() in [b.lower() for b in brands]
        ):
            filtered.append(item)

    filtered = sorted(filtered, key=lambda x: x["price"])
    return json.dumps(filtered[:5], indent=2)


In [23]:
# Tool 2: Inventory Check
inventory_db = {
    "P1001": True,
    "P1002": True,
    "P1003": False,
    "P1004": True,
    "P1005": False,
    "P1006": True,
    "P1007": False,
    "P1008": True,
    "P1009": True,
    "P1010": False
}
def inventory_check_tool(input_ids: str):
    try:
        ids = json.loads(input_ids)
    except Exception:
        return "Invalid product ID list."

    result = {}
    for pid in ids:
        result[pid] = "In Stock ✅" if inventory_db.get(pid, False) else "Out of Stock ❌"
    return json.dumps(result, indent=2)


In [24]:
# Set up tools
tools = [
    Tool(
        name="FilterCatalog",
        func=filter_products_tool,
        description="Filters products by price, size, brand preferences"
    ),
    Tool(
        name="InventoryCheck",
        func=inventory_check_tool,
        description="Checks availability status of product IDs"
    )
]

In [25]:
# Initialize the agent
llm = ChatOpenAI(model="gpt-4", temperature=0.3)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [29]:
# Run the agent
query = "Find me running shoes under ₹5,000 in size 9 from Nike or Adidas."
print("\n🧠 Query:", query)
response = agent.run(query)

print("\n📦 Final Result:")
print(response)


🧠 Query: Find me running shoes under ₹5,000 in size 9 from Nike or Adidas.


> Entering new AgentExecutor chain...
I need to filter the catalog based on the user's preferences.
Action: FilterCatalog
Action Input: {"type": "running shoes", "price": "<5000", "size": "9", "brand": ["Nike", "Adidas"]}
Observation: [
  {
    "id": "P1006",
    "name": "Adidas Runfalcon",
    "brand": "Adidas",
    "price": 4499,
    "size_available": [
      9
    ],
    "category": "Running Shoes"
  },
  {
    "id": "P1001",
    "name": "Nike Revolution 6",
    "brand": "Nike",
    "price": 4599,
    "size_available": [
      7,
      8,
      9
    ],
    "category": "Running Shoes"
  },
  {
    "id": "P1002",
    "name": "Adidas Galaxy 6",
    "brand": "Adidas",
    "price": 4999,
    "size_available": [
      9,
      10
    ],
    "category": "Running Shoes"
  }
]
Thought:I have found three pairs of shoes that match the user's preferences. Now I need to check if they are available in the inventory.
Ac